In [ ]:
# 필요 라이브러리 임포트
import firebase_admin
from firebase_admin import credentials, db
import pandas as pd
import neurokit2 as nk
import matplotlib.pyplot as plt
import numpy as np
import json

# 기존 Firebase 앱이 있으면 삭제하고 새로 초기화
if firebase_admin._apps:
    firebase_admin.delete_app(firebase_admin.get_app())

# Firebase 앱 초기화 (사용자의 키와 URL로 수정 필요)
cred = credentials.Certificate("hrvdataset-firebase-adminsdk-oof96-2a96d6ac7f.json")  # Firebase Admin SDK JSON 파일
firebase_admin.initialize_app(cred, {"databaseURL": "https://hrvdataset-default-rtdb.firebaseio.com/"})

# 데이터 가져오기
# ref = db.reference("HeartRateData")
# data = ref.get()

# wakeup.json 파일에서 데이터 불러오기
with open("gosleep.json", "r") as f:
    data_json = json.load(f)


# 데이터 정리
ppg_values = []
timestamps = []

for key, value in data_json["HeartRateData"].items():
    if not value["isError"]:  # 오류 없는 데이터만 사용
        ppg_values.append(value["ppgGreen"])
        timestamps.append(pd.to_datetime(value["timestamp"]))

# 데이터프레임 생성 및 정렬
df = pd.DataFrame({"Timestamp": timestamps, "PPG": ppg_values})
df = df.sort_values("Timestamp")

# 초반 2초 데이터 삭제
start_time = df["Timestamp"].iloc[0]
df = df[df["Timestamp"] > start_time + pd.Timedelta(seconds=2)]

# PPG 신호 청소 (샘플링 주파수 25Hz)
fs = 25
ppg_cleaned = nk.ppg_clean(df["PPG"], sampling_rate=fs)

# 피크 검출 함수 (min_y 추가)
def find_prominent_peaks(signal, threshold=0.1, min_y=0):
    peaks = []
    for i in range(1, len(signal) - 1):
        if signal[i] > signal[i - 1] and signal[i] > signal[i + 1] and signal[i] > min_y:
            left_min = min(signal[max(0, i - 5):i])
            right_min = min(signal[i + 1:i + 6])
            prominence = signal[i] - max(left_min, right_min)
            if prominence > threshold:
                peaks.append(i)
    return peaks

# 피크 검출
min_y = 0
peaks_indices = find_prominent_peaks(ppg_cleaned, threshold=0.1, min_y=min_y)


# 피크의 타임스탬프와 값 추출
peaks_timestamps = df["Timestamp"].iloc[peaks_indices].values
peaks_values = ppg_cleaned[peaks_indices]

# RR 간격 계산 (밀리초 단위)
rr_intervals = np.diff(peaks_timestamps).astype('timedelta64[ms]').astype(int)

# RR 간격을 다시 피크 인덱스로 변환
peaks_from_rr = nk.intervals_to_peaks(rr_intervals)

# PPG 피크 검출
peaks, info = nk.ppg_peaks(ppg_cleaned, sampling_rate=fs, show=True)

# peaks 데이터프레임의 인덱스를 df와 일치시키기
peaks["Timestamp"] = df["Timestamp"].reset_index(drop=True)

# HRV 도메인별 분석
hrv_time_metrics = nk.hrv_time(peaks_from_rr, sampling_rate=1000)
hrv_frequency_metrics = nk.hrv_frequency(peaks_from_rr, sampling_rate=1000)
hrv_nonlinear_metrics = nk.hrv_nonlinear(peaks_from_rr, sampling_rate=1000)

# HRV 지표 출력
print("HRV Time-Domain Metrics:")
print(hrv_time_metrics)
print("\nHRV Frequency-Domain Metrics:")
print(hrv_frequency_metrics)
print("\nHRV Non-Linear Metrics:")
print(hrv_nonlinear_metrics)


# 시각화
plt.figure(figsize=(10, 5))
plt.plot(df["Timestamp"], ppg_cleaned, label="Cleaned PPG", color="blue")
plt.scatter(peaks_timestamps, peaks_values, color="red", label="Peaks", zorder=3)
plt.legend()
plt.xlabel("Time")
plt.ylabel("PPG Signal")
plt.title("PPG Signal with Detected Peaks")
plt.xticks(rotation=45)
plt.grid()
plt.show()


In [ ]:
import firebase_admin
from firebase_admin import credentials, db
import pandas as pd
import neurokit2 as nk
import matplotlib.pyplot as plt
import numpy as np
import json
from scipy.stats import chi2, f
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from scipy.stats import chi2, f
import numpy as np
import matplotlib.pyplot as plt

# ---------------------------
# 0. Firebase 앱 초기화 및 데이터 로드
# ---------------------------

# 기존 Firebase 앱 초기화 (필요 시)
if firebase_admin._apps:
    firebase_admin.delete_app(firebase_admin.get_app())

# Firebase 앱 초기화 (사용자의 키와 URL로 수정 필요)
cred = credentials.Certificate("hrvdataset-firebase-adminsdk-oof96-2a96d6ac7f.json")
firebase_admin.initialize_app(cred, {"databaseURL": "https://hrvdataset-default-rtdb.firebaseio.com/"})

# 데이터 가져오기
# ref = db.reference("HeartRateData")
# data = ref.get()

# gosleep.json 파일에서 데이터 불러오기 (firebase와 동일한 형식)
with open("test.json", "r") as file_in:
    data_json = json.load(file_in)

# ---------------------------
# 1. PPG 데이터 처리 및 HRV 분석 (기존 방식)
# ---------------------------

# 데이터 정리 (오류 없는 데이터만 사용)
ppg_values = []
ppg_timestamps = []
for key, value in data_json["HeartRateData"].items():
    if not value["isError"]:
        ppg_values.append(value["ppgGreen"])
        ppg_timestamps.append(pd.to_datetime(value["timestamp"]))

# 데이터프레임 생성 및 타임스탬프 기준 정렬
df_ppg = pd.DataFrame({"Timestamp": ppg_timestamps, "PPG": ppg_values})
df_ppg = df_ppg.sort_values("Timestamp")

# 초반 2초 데이터 삭제 (시작 안정화)
start_time = df_ppg["Timestamp"].iloc[0]
df_ppg = df_ppg[df_ppg["Timestamp"] > start_time + pd.Timedelta(seconds=2)]

# PPG 신호 청소 (샘플링 주파수 25Hz)
fs = 25
ppg_cleaned = nk.ppg_clean(df_ppg["PPG"], sampling_rate=fs)

# 피크 검출 함수 (min_y 추가)
def find_prominent_peaks(signal, threshold=0.1, min_y=0):
    peaks = []
    for i in range(1, len(signal) - 1):
        if signal[i] > signal[i - 1] and signal[i] > signal[i + 1] and signal[i] > min_y:
            left_min = min(signal[max(0, i - 5):i])
            right_min = min(signal[i + 1:i + 6])
            prominence = signal[i] - max(left_min, right_min)
            if prominence > threshold:
                peaks.append(i)
    return peaks

# 전체 신호에 대해 피크 검출
peaks_indices = find_prominent_peaks(ppg_cleaned, threshold=0.1, min_y=0)
peaks_timestamps = df_ppg["Timestamp"].iloc[peaks_indices].values  # numpy array of timestamps
peaks_values = ppg_cleaned[peaks_indices]

# 전체 신호 및 피크 시각화 (옵션)
plt.figure(figsize=(10, 5))
plt.plot(df_ppg["Timestamp"], ppg_cleaned, label="Cleaned PPG", color="blue")
plt.scatter(peaks_timestamps, peaks_values, color="red", label="Peaks", zorder=3)
plt.legend()
plt.xlabel("Time")
plt.ylabel("PPG Signal")
plt.title("PPG Signal with Detected Peaks")
plt.xticks(rotation=45)
plt.grid()
plt.show()

# ---------------------------
# 2. Accelerometer 데이터 처리 및 이상 탐지 (MSPC-PCA)
# ---------------------------

# Accelerometer 데이터 읽기 및 DataFrame 생성
accel_values = []
accel_timestamps = []
for key, value in data_json["AccelerometerData"].items():
    # 실제 키 이름("ax", "ay", "az")는 데이터 구조에 맞게 수정하세요.
    accel_values.append([value["ax"], value["ay"], value["az"]])
    accel_timestamps.append(pd.to_datetime(value["timestamp"]))
df_accel = pd.DataFrame(accel_values, columns=["ax", "ay", "az"])
df_accel["Timestamp"] = accel_timestamps
df_accel = df_accel.sort_values("Timestamp")

# PPG와 동일하게 초반 2초 데이터 삭제 (시작 안정화)
start_time = df_accel["Timestamp"].iloc[0]
df_accel = df_accel[df_accel["Timestamp"] > start_time + pd.Timedelta(seconds=2)]

# 인덱스로 설정
df_accel.set_index("Timestamp", inplace=True)

# 각 2분 윈도우마다 가속도계 데이터 대표값 도출 (평균 vs. 중앙값, CV 기준)
def compute_feature(series):
    mean_val = series.mean()
    std_val = series.std()
    cv = (std_val/mean_val * 100) if mean_val != 0 else 0
    return series.mean() if cv < 33 else series.median()

accel_features = []
accel_intervals = []
for window_start, group in df_accel.groupby(pd.Grouper(freq="2Min")):
    if group.empty:
        continue
    window_end = window_start + pd.Timedelta(minutes=2)
    feat_x = compute_feature(group["ax"])
    feat_y = compute_feature(group["ay"])
    feat_z = compute_feature(group["az"])
    accel_features.append([feat_x, feat_y, feat_z])
    accel_intervals.append((window_start, window_end))
df_accel_features = pd.DataFrame(accel_features, columns=["feat_x", "feat_y", "feat_z"])
df_accel_features["WindowStart"] = [interval[0] for interval in accel_intervals]
df_accel_features["WindowEnd"] = [interval[1] for interval in accel_intervals]

# MSPC-PCA 이상 탐지: 단일 주성분 PCA를 사용해 Hotelling T²와 SPE 계산
alpha = 0.05

N_accel = len(df_accel_features)
X_accel = df_accel_features[["feat_x", "feat_y", "feat_z"]].values.astype(float)
X_accel_scaled = StandardScaler().fit_transform(X_accel)
pca_accel = PCA(n_components=1)
scores_accel = pca_accel.fit_transform(X_accel_scaled).flatten()
variance_accel = pca_accel.explained_variance_[0]
T2_accel = (scores_accel**2) / variance_accel
ulc_t2_accel = (1 * (N_accel + 1) * (N_accel - 1) / (N_accel * (N_accel - 1))) * f.ppf(1 - alpha, 1, N_accel - 1)
X_hat_accel = pca_accel.inverse_transform(scores_accel.reshape(-1,1))
SPE_accel = ((X_accel_scaled - X_hat_accel)**2).sum(axis=1)
b_accel, v_accel = SPE_accel.mean(), SPE_accel.var()
df_chi_accel = (2 * b_accel * b_accel) / v_accel
ulc_spe_accel = (v_accel / (2 * b_accel)) * chi2.ppf(1 - alpha, df_chi_accel)

df_accel_features["T2"] = T2_accel
df_accel_features["SPE"] = SPE_accel
df_accel_features["OOC_T2"] = T2_accel > ulc_t2_accel
df_accel_features["OOC_SPE"] = SPE_accel > ulc_spe_accel
df_accel_features["Anomaly"] = df_accel_features["OOC_T2"] | df_accel_features["OOC_SPE"]

# 이상 구간(2분 윈도우) 기록: 이상치인 윈도우의 시작, 종료 시간
accel_anomaly_intervals = []
for index, row in df_accel_features.iterrows():
    if row["Anomaly"]:
        accel_anomaly_intervals.append((row["WindowStart"], row["WindowEnd"]))
print("Accelerometer anomaly intervals:", accel_anomaly_intervals)

# Control Chart 시각화 — Hotelling’s T² (Accelerometer)
plt.figure()
plt.plot(df_accel_features["WindowStart"], T2_accel, marker="o")
plt.axhline(ulc_t2_accel, linestyle="--", color="red", label="ULC T²")
plt.title("Accelerometer Domain — Hotelling’s T²")
plt.xlabel("Window Start")
plt.ylabel("T²")
plt.xticks(rotation=45)
plt.legend()
plt.tight_layout()
plt.show()

# Control Chart 시각화 — SPE (Accelerometer)
plt.figure()
plt.plot(df_accel_features["WindowStart"], SPE_accel, marker="o")
plt.axhline(ulc_spe_accel, linestyle="--", color="red", label="ULC SPE")
plt.title("Accelerometer Domain — SPE")
plt.xlabel("Window Start")
plt.ylabel("SPE")
plt.xticks(rotation=45)
plt.legend()
plt.tight_layout()
plt.show()

# 이상 구간 출력 (Accelerometer)
ooc_accel = df_accel_features[df_accel_features["OOC_T2"] | df_accel_features["OOC_SPE"]]
print("\n=== Accelerometer Domain Out‑of‑Control Segments ===")
print(ooc_accel[["WindowStart", "WindowEnd", "OOC_T2", "OOC_SPE"]])

# ---------------------------
# 3. PPG 데이터 필터링: accelerometer 이상 구간 제거
# ---------------------------
def is_in_anomaly(timestamp, anomaly_intervals):
    for start, end in anomaly_intervals:
        if start <= timestamp <= end:
            return True
    return False

df_ppg_filtered = df_ppg[~df_ppg["Timestamp"].apply(lambda t: is_in_anomaly(t, accel_anomaly_intervals))]
print(f"Original PPG count: {len(df_ppg)}, Filtered count: {len(df_ppg_filtered)}")

# ---------------------------
# 4. PPG 신호 전처리: 필터링된 PPG 데이터로 HRV 및 MSPC-PCA 이상 탐지 수행
# ---------------------------
ppg_cleaned_filtered = nk.ppg_clean(df_ppg_filtered["PPG"], sampling_rate=fs)
peaks_indices_filtered = find_prominent_peaks(ppg_cleaned_filtered, threshold=0.1, min_y=0)
peaks_timestamps_filtered = df_ppg_filtered["Timestamp"].iloc[peaks_indices_filtered].values
peaks_values_filtered = ppg_cleaned_filtered[peaks_indices_filtered]

plt.figure(figsize=(10,5))
plt.plot(df_ppg_filtered["Timestamp"], ppg_cleaned_filtered, label="Cleaned PPG (Filtered)", color="blue")
plt.scatter(peaks_timestamps_filtered, peaks_values_filtered, color="red", label="Peaks (Filtered)", zorder=3)
plt.legend()
plt.xlabel("Time")
plt.ylabel("PPG Signal")
plt.title("PPG Signal with Detected Peaks (Filtered by Accelerometer Anomalies)")
plt.xticks(rotation=45)
plt.grid()
plt.show()

# ----------------------------------------------------
# 5. 피크 기준으로 2분 단위 HRV 분석
# ----------------------------------------------------
hrv_results = []
i = 0
peaks_ts = peaks_timestamps_filtered
while i < len(peaks_ts):
    seg_start = peaks_ts[i]
    seg_end = seg_start + pd.Timedelta(minutes=2)
    segment_peaks = []
    while i < len(peaks_ts) and peaks_ts[i] < seg_end:
        segment_peaks.append(peaks_indices_filtered[i])
        i += 1
    if len(segment_peaks) < 2:  
        print(f"구간 {seg_start} ~ {seg_end} 에는 피크가 부족하여 HRV 계산을 건너뜁니다.")
        continue
    
    # HR 시리즈 생성 (bpm)
    segment_times = peaks_ts[i - len(segment_peaks):i]  # datetime64 array
    rri = np.diff(segment_times).astype("timedelta64[ms]").astype(int)  # ms 단위 RR intervals
    hr_series = 60000 / rri
    
    hrv_time = nk.hrv_time(segment_peaks, sampling_rate=25)
    hrv_time_metrics = {
        "mean_nni": hrv_time["HRV_MeanNN"].iloc[0],
        "median_nni": hrv_time["HRV_MedianNN"].iloc[0],
        "range_nni": hrv_time["HRV_MaxNN"].iloc[0] - hrv_time["HRV_MinNN"].iloc[0],
        "sdnn": hrv_time["HRV_SDNN"].iloc[0],
        "sdsd": hrv_time["HRV_SDSD"].iloc[0],
        "rmssd": hrv_time["HRV_RMSSD"].iloc[0],
        "nni_50": int(np.sum(np.abs(np.diff(rri)) > 50)),
        "pnni_50": hrv_time["HRV_pNN50"].iloc[0],
        "nni_20": int(np.sum(np.abs(np.diff(rri)) > 20)),
        "pnni_20": hrv_time["HRV_pNN20"].iloc[0],
        "cvsd": hrv_time["HRV_CVSD"].iloc[0],
        "cvnni": hrv_time["HRV_CVNN"].iloc[0],
        "mean_hr": np.nanmean(hr_series),
        "min_hr": np.nanmin(hr_series),
        "max_hr": np.nanmax(hr_series),
        "std_hr": np.nanstd(hr_series, ddof=1),
    }
    
    # 주파수 영역 HRV 계산
    hrv_freq = nk.hrv_frequency(segment_peaks, sampling_rate=25, normalize=False)
    if not hrv_freq.empty:
        hrv_freq_metrics = {
            "power_vlf": hrv_freq["HRV_VLF"].iloc[0],
            "power_lf":  hrv_freq["HRV_LF"].iloc[0],
            "power_hf":  hrv_freq["HRV_HF"].iloc[0],
            "total_power": hrv_freq["HRV_TP"].iloc[0],
            "lf_hf_ratio": hrv_freq["HRV_LFHF"].iloc[0]
        }
    else:
        hrv_freq_metrics = {k: np.nan for k in ["power_vlf","power_lf","power_hf","total_power","lf_hf_ratio"]}
    
    # 비선형 영역 주파수 계산
    hrv_nonlinear = nk.hrv_nonlinear(segment_peaks, sampling_rate=25)
    if not hrv_nonlinear.empty:
        hrv_nonlinear_metrics = {
            "csi":            hrv_nonlinear["HRV_CSI"].iloc[0],
            "cvi":            hrv_nonlinear["HRV_CVI"].iloc[0],
            "modified_csi":   hrv_nonlinear["HRV_CSI_Modified"].iloc[0],
            "sampen":         hrv_nonlinear["HRV_SampEn"].iloc[0],
        }
    else:
        hrv_nonlinear_metrics = {k: np.nan for k in ["csi","cvi","modified_csi","sampen"]}
    
    
    result = {
        "Segment Start": seg_start,
        "Segment End": seg_end,
        "HRV Time Metrics": hrv_time_metrics,
        "HRV Frequency Metrics": hrv_freq_metrics,
        "HRV Nonlinear Metrics": hrv_nonlinear_metrics
    }
    hrv_results.append(result)

# HRV 결과를 하나의 DataFrame으로 결합하기
results_list = []
for res in hrv_results:
    row = {
        "Segment Start": res["Segment Start"],
        "Segment End":   res["Segment End"]
    }
    # Time-domain dict → Time_ 접두어
    for key, val in res["HRV Time Metrics"].items():
        row[f"Time_{key}"] = val

    # Frequency-domain dict → Freq_ 접두어
    for key, val in res["HRV Frequency Metrics"].items():
        row[f"Freq_{key}"] = val

    # Nonlinear-domain dict → Nonlinear_ 접두어
    for key, val in res["HRV Nonlinear Metrics"].items():
        row[f"Nonlinear_{key}"] = val

    results_list.append(row)

df_hrv_results = pd.DataFrame(results_list)

# --- 딕셔너리 → DataFrame 생성 ---
df_segment = pd.DataFrame({
    "Time": pd.Series(res["HRV Time Metrics"]),
    "Frequency": pd.Series(res["HRV Frequency Metrics"]),
    "Nonlinear": pd.Series(res["HRV Nonlinear Metrics"])
})


start_str = pd.to_datetime(seg_start).strftime("%H:%M:%S")
end_str   = pd.to_datetime(seg_end).strftime("%H:%M:%S")
# 출력
print(f"\n===== Segment {start_str} → {end_str} =====")
print(df_segment.round(3))


# HRV 결과 CSV 파일로 저장하려면 여기 사용용
df_hrv_results.to_csv("hrv_results.csv", index=False)
print("HRV 결과가 hrv_results.csv 파일에 저장되었습니다.")

# ---------------------------
# 6. MSPC-PCA 이상 탐지 on HRV (각 도메인별)
# ---------------------------



N = len(df_hrv_results)

# 도메인별 컬럼 매핑
domains = {
    "Time": df_hrv_results.filter(regex="^Time_").columns,
    "Freq": df_hrv_results.filter(regex="^Freq_").columns,
    "Nonlinear": df_hrv_results.filter(regex="^Nonlinear_").columns
}

for domain, cols in domains.items():
    # 1) 데이터 준비 및 표준화
    X = df_hrv_results[cols].fillna(0).values.astype(float)
    X_scaled = StandardScaler().fit_transform(X)

    # 2) PCA (단일 주성분)
    pca = PCA(n_components=1)
    scores = pca.fit_transform(X_scaled).flatten()
    variance = pca.explained_variance_[0]

    # 3) Hotelling’s T² 계산
    T2 = (scores ** 2) / variance
    ulc_t2 = (1 * (N + 1) * (N - 1) / (N * (N - 1))) * f.ppf(1 - alpha, 1, N - 1)

    # 4) SPE (Q‑statistic) 계산
    X_hat = pca.inverse_transform(scores.reshape(-1, 1))
    SPE = ((X_scaled - X_hat) ** 2).sum(axis=1)
    b, v = SPE.mean(), SPE.var()
    df_chi = (2 * b * b) / v
    ulc_spe = (v / (2 * b)) * chi2.ppf(1 - alpha, df_chi)

    # 5) 결과 DataFrame 추가
    df_hrv_results[f"{domain}_T2"] = T2
    df_hrv_results[f"{domain}_SPE"] = SPE
    df_hrv_results[f"{domain}_OOC_T2"] = T2 > ulc_t2
    df_hrv_results[f"{domain}_OOC_SPE"] = SPE > ulc_spe

    # 6) Control Chart 시각화 — Hotelling’s T²
    plt.figure()
    plt.plot(df_hrv_results["Segment Start"], T2, marker="o")
    plt.axhline(ulc_t2, linestyle="--")
    plt.title(f"{domain} Domain — Hotelling’s T²")
    plt.xlabel("Segment Start")
    plt.ylabel("T²")
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

    # 7) Control Chart 시각화 — SPE
    plt.figure()
    plt.plot(df_hrv_results["Segment Start"], SPE, marker="o")
    plt.axhline(ulc_spe, linestyle="--")
    plt.title(f"{domain} Domain — SPE")
    plt.xlabel("Segment Start")
    plt.ylabel("SPE")
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

    # 8) 이상구간 출력
    ooc = df_hrv_results[df_hrv_results[f"{domain}_OOC_T2"] | df_hrv_results[f"{domain}_OOC_SPE"]]
    print(f"\n=== {domain} Domain Out‑of‑Control Segments ===")
    print(ooc[["Segment Start","Segment End",f"{domain}_OOC_T2",f"{domain}_OOC_SPE"]])



In [ ]:
# neourokit2를 사용해 peak를 구하는 함수 peak는 정확하지만 탐지하지 못하는 peak가 있음
# 필요 라이브러리 임포트
import firebase_admin
from firebase_admin import credentials, db
import pandas as pd
import neurokit2 as nk
import matplotlib.pyplot as plt

if firebase_admin._apps:
    firebase_admin.delete_app(firebase_admin.get_app())

# 🔹 Firebase 앱 새로 초기화
cred = credentials.Certificate("hrvdataset-firebase-adminsdk-oof96-2a96d6ac7f.json")  # Firebase Admin SDK JSON 파일
firebase_admin.initialize_app(cred, {"databaseURL": "https://hrvdataset-default-rtdb.firebaseio.com/"})

ref = db.reference("HeartRateData")
data = ref.get()

# 데이터 정리
ppg_values = []
timestamps = []

for key, value in data.items():
    if not value["isError"]:  # 오류 없는 데이터만 사용
        ppg_values.append(value["ppgGreen"])
        timestamps.append(pd.to_datetime(value["timestamp"]))

# 데이터프레임 생성
df = pd.DataFrame({"Timestamp": timestamps, "PPG": ppg_values})
df = df.sort_values("Timestamp")  # 시간 순 정렬

# 초반 2초의 데이터를 삭제
start_time = df["Timestamp"].iloc[0]
df = df[df["Timestamp"] > start_time + pd.Timedelta(seconds=2)]

# PPG 신호 처리 (25Hz 가정)
fs = 25  # 샘플링 주파수
ppg_cleaned = nk.ppg_clean(df["PPG"], sampling_rate=fs)

# PPG 피크 검출
peaks, info = nk.ppg_peaks(ppg_cleaned, sampling_rate=fs)

# peaks 데이터프레임의 인덱스를 df와 일치시키기
peaks["Timestamp"] = df["Timestamp"].reset_index(drop=True)

# 시각화
plt.figure(figsize=(10, 5))
plt.plot(df["Timestamp"], ppg_cleaned, label="Cleaned PPG", color="blue")
plt.scatter(peaks["Timestamp"][peaks["PPG_Peaks"] == 1], ppg_cleaned[peaks["PPG_Peaks"] == 1], color="red", label="Peaks", zorder=3)
plt.legend()
plt.xlabel("Time")
plt.ylabel("PPG Signal")
plt.title("PPG Signal with Detected Peaks")
plt.xticks(rotation=45)
plt.grid()
plt.show()
